In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import arviz as az
import pymc as pm
import corner
import scipy.stats as st
from statistics import mean
from collections import Counter

In [2]:
file_path = 'Datasets/mt_genes_metadata.csv'

met = pd.read_csv(file_path)

pro_code=met[met['gene_type']=='protein_coding']
intpro=pro_code['gene_name'].tolist()
intpro=['MT-ND1','MT-ND2','MT-ND3','MT-ND4','MT-ND4L','MT-ND5','MT-ND6', 'MT-CO1', 'MT-CO2','MT-CO3', 'MT-ATP6', 'MT-ATP8', 'MT-CYB']
intpro

['MT-ND1',
 'MT-ND2',
 'MT-ND3',
 'MT-ND4',
 'MT-ND4L',
 'MT-ND5',
 'MT-ND6',
 'MT-CO1',
 'MT-CO2',
 'MT-CO3',
 'MT-ATP6',
 'MT-ATP8',
 'MT-CYB']

In [3]:
file_path = 'Datasets/Donor1_CD4_Genes.csv'

p14 = pd.read_csv(file_path)

p14.drop(columns=p14.columns[0], axis=1, inplace=True)

pat1_4int=pd.DataFrame([p14[i] for i in intpro]).T

file_path = 'Datasets/Donor1_CD8_Genes.csv'

p18 = pd.read_csv(file_path)

p18.drop(columns=p18.columns[0], axis=1, inplace=True)

pat1_8int=pd.DataFrame([p18[i] for i in intpro]).T

file_path = 'Datasets/Donor2_CD4_Genes.csv'

p24 = pd.read_csv(file_path)

p24.drop(columns=p24.columns[0], axis=1, inplace=True)

pat2_4int=pd.DataFrame([p24[i] for i in intpro]).T

file_path = 'Datasets/Donor2_CD8_Genes.csv'

p28 = pd.read_csv(file_path)

p28.drop(columns=p28.columns[0], axis=1, inplace=True)

pat2_8int=pd.DataFrame([p28[i] for i in intpro]).T

total=pd.concat([p14,p18,p24,p28])

total_int=pd.concat([pat1_4int,pat1_8int, pat2_4int,pat2_8int])

totaln=len(total_int)


In [4]:
total_sampto=total.sum() #total number of each gene
totalgene=total_sampto.sum() #total number of genes

In [6]:
#number of DNA strands in each file
p14n=len(pat1_4int)
p18n=len(pat1_8int)
p24n=len(pat2_4int)
p28n=len(pat2_8int)

In [12]:
#Calculates the number of genes for each DNA
p14_allsampto=p14.sum(axis=1)
p14_allsampto

0       1547
1       2578
2       2922
3       2735
4       4317
        ... 
2883    3218
2884    3065
2885    4664
2886    5669
2887    4441
Length: 2888, dtype: int64

Normalising data and then computing Unbiased estimate of neg binom proportion

In [40]:
r=1000
uem14_list=[]
for j in intpro:
    uem_sample_list=[]
    for i in range(p14n):   
        norm_cons=(p14_allsampto[i]-p14[j][i])/r #inidividual normalisation constant
        x=p14[j][i]/norm_cons
        uem=(r-1)/(x+r-1)
        uem_sample_list.append(uem)
    uem14_list.append(mean(uem_sample_list))
uem14_list=dict(zip(intpro,uem14_list))
uem14_list

{'MT-ND1': 0.9982130218641465,
 'MT-ND2': 0.9989420198086033,
 'MT-ND3': 0.9987206822526757,
 'MT-ND4': 0.9992367635190114,
 'MT-ND4L': 0.9970263015502824,
 'MT-ND5': 0.9988379600574597,
 'MT-ND6': 0.9997615269486313,
 'MT-CO1': 0.9954579488639743,
 'MT-CO2': 0.9922420526743582,
 'MT-CO3': 0.993906312992069,
 'MT-ATP6': 0.9995115561342489,
 'MT-ATP8': 0.9956107948395834,
 'MT-CYB': 0.9952249786447018}

χ² test:

In [57]:
#p1_4
df14=p14n-1
cv=st.chi2.ppf(0.95,df14)
chi1_p14_ts=[]
chi1_p14_res=[]
for j in intpro:
    ts=0
    p=uem14_list[j]
    for i in range(p14n):
        r=1000
        norm_cons=(p14_allsampto[i]-p14[j][i])/r #inidividual normalisation constant
        o=p14[j][i]/norm_cons
        e=r*(1-p)/p
        ts+=(e-o)**2/e
    chi1_p14_ts.append(ts)
    chi1_p14_res.append(ts-cv)
chi1_p14_res

[-205.50701735868915,
 -940.9417973989403,
 -1056.8396417059225,
 -1331.4283159478762,
 904.9347990797951,
 -1103.3607190119499,
 -1508.307789281577,
 3613.4572403626253,
 5023.164590000371,
 3341.428344759509,
 -1785.7717366269746,
 3261.264572957297,
 1903.1279271571675]

Below is unrefined stuff

In [71]:
#p1_4
df14=p14n-2
cv=st.chi2.ppf(0.95,df14)
chi2_p14_ts=[]
chi2_p14_res=[]
for j in intpro:
    ts=0
    p=uem14t_list[j]
    for i in range(p14n):
        if p14_allsampto[i]==0:
            ts+=0
        else:
            r=p14_allsampto[i]-pat1_4int[j][i]
            o=pat1_4int[j][i]
            e=r*(1-p)/p
            ts+=(e-o)**2/e
    chi2_p14_ts.append(ts)
    chi2_p14_res.append(ts-cv)
chi2_p14_res

[4111.197925329632,
 3043.734773332208,
 2419.2040819217004,
 1987.8232946176636,
 6403.878663851136,
 2852.984004617039,
 1548.3527769126326,
 13955.610254847617,
 15100.751960680309,
 11456.024761625824,
 984.9264501561092,
 11442.159314060757,
 9233.692961764658]